In [1]:
from glob2 import glob
fns = glob('/scratch/PI/cqf/har_data/ucf101/videos/*.avi')

In [2]:
def mwlines(lines, fname):
    with open(fname, 'w') as fout:
        fout.write('\n'.join(lines))

In [3]:
mwlines(fns, 'ucf101_videos.list')

In [1]:
l1 = [1, 2, 3]
l2 = [4, 5, 6]
z = list(zip(l1, l2))
z

[(1, 4), (2, 5), (3, 6)]

In [3]:
z[0:2]

[(1, 4), (2, 5)]

In [4]:
z[0:2][0]

(1, 4)

In [20]:
import pickle

with open('/home/zpengac/pose/PoseSegmentationMask/ntu60_hrnet.pkl', 'rb') as fin:
    data = pickle.load(fin)

data.keys()

dict_keys(['split', 'annotations'])

In [21]:
data['annotations'][0]['frame_dir']

'S001C001P001R001A001'

In [12]:
data['annotations'][0]['keypoint'].shape

(1, 119, 17, 2)

In [16]:
import torch
import numpy as np
skl = torch.from_numpy(data['annotations'][0]['keypoint'])
y_mins = torch.min(skl[..., 1], dim=-1)[0].type(torch.int)
print(y_mins.shape)

torch.Size([1, 119])


In [17]:
data['annotations'][0]['total_frames']

119

In [19]:
sd = torch.load('/home/zpengac/pose/PoseSegmentationMask/logs/jhmdb_final5/best_PCK_epoch_100.pth', map_location='cpu')
sd['state_dict'].keys()

odict_keys(['backbone.conv1.weight', 'backbone.bn1.weight', 'backbone.bn1.bias', 'backbone.bn1.running_mean', 'backbone.bn1.running_var', 'backbone.bn1.num_batches_tracked', 'backbone.conv2.weight', 'backbone.bn2.weight', 'backbone.bn2.bias', 'backbone.bn2.running_mean', 'backbone.bn2.running_var', 'backbone.bn2.num_batches_tracked', 'backbone.layer1.0.conv1.weight', 'backbone.layer1.0.bn1.weight', 'backbone.layer1.0.bn1.bias', 'backbone.layer1.0.bn1.running_mean', 'backbone.layer1.0.bn1.running_var', 'backbone.layer1.0.bn1.num_batches_tracked', 'backbone.layer1.0.conv2.weight', 'backbone.layer1.0.bn2.weight', 'backbone.layer1.0.bn2.bias', 'backbone.layer1.0.bn2.running_mean', 'backbone.layer1.0.bn2.running_var', 'backbone.layer1.0.bn2.num_batches_tracked', 'backbone.layer1.0.conv3.weight', 'backbone.layer1.0.bn3.weight', 'backbone.layer1.0.bn3.bias', 'backbone.layer1.0.bn3.running_mean', 'backbone.layer1.0.bn3.running_var', 'backbone.layer1.0.bn3.num_batches_tracked', 'backbone.layer1

In [23]:
def read_psm(psm_path):
    # PSM is stored in a PNG image
    canvas = np.array(Image.open(psm_path)).squeeze()

    # Attributes are stored in the last row
    J = canvas[-1, -1]
    H = canvas[-1, -2]
    W = canvas[-1, -3]
    canvas[-1, -1] = 0
    canvas[-1, -2] = 0
    canvas[-1, -3] = 0
    nw = 4
    nh = int(np.ceil(J / nw))

    # Split the canvas into masks
    out_masks = np.zeros((J, H, W), dtype=np.uint8)
    for i in range(J):
        x = (i % nw) * W
        y = (i // nw) * H
        out_masks[i] = canvas[y:y+H, x:x+W]
    #out_masks = torch.from_numpy(out_masks) > 0
    
    return out_masks

In [27]:
from glob2 import glob
from tqdm import tqdm
from PIL import Image
import os
from multiprocessing import Pool

fns = glob('/scratch/PI/cqf/har_data/ucf101/psm/*/*.png')

def run(fn):
    psm = np.array(Image.open(fn)).squeeze()
    J = psm[-1, -1]
    H = psm[-1, -2]
    W = psm[-1, -3]
    psm_new = ((psm.astype(np.float32) / 255.) > 0.5).astype(np.uint8) * 255
    psm_new[-1, -1] = J
    psm_new[-1, -2] = H
    psm_new[-1, -3] = W
    fn_new = fn.replace('psm', 'psm2')
    os.makedirs(os.path.dirname(fn_new), exist_ok=True)
    if not os.path.exists(fn_new):
        Image.fromarray(psm_new).save(fn_new)

with Pool(16) as p:
    list(tqdm(p.imap(run, fns), total=len(fns)))

100%|██████████| 1535412/1535412 [44:38<00:00, 573.26it/s] 


In [5]:
from glob2 import glob
fns = glob('/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S001C001P002R001A007_rgb.avi')
print(len(fns))

1


In [3]:
print(fns)

['/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S003C001P017R002A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S011C003P019R002A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S010C002P015R001A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S008C002P029R002A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S002C003P008R001A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S014C001P027R002A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S011C001P038R001A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S016C001P008R001A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S003C002P016R001A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S006C001P024R001A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S007C002P026R002A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S004C003P020R001A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+d_rgb/S013C002P017R001A023_rgb.avi', '/scratch/PI/cqf/har_data/ntu/nturgb+

In [6]:
import pickle

with open('/home/zpengac/pose/pyskl/data/ntu60_hrnet_psm.pkl', 'rb') as fin:
    data = pickle.load(fin)

data['annotations'][126].keys()

dict_keys(['frame_dir', 'label', 'start_index', 'all_frame_dirs', 'total_frames', 'img_shape', 'original_shape'])

In [8]:
print(data['annotations'][126]['all_frame_dirs'])

['S001C001P002R001A007_rgb/040.png', 'S001C001P002R001A007_rgb/006.png', 'S001C001P002R001A007_rgb/042.png', 'S001C001P002R001A007_rgb/069.png', 'S001C001P002R001A007_rgb/002.png', 'S001C001P002R001A007_rgb/046.png', 'S001C001P002R001A007_rgb/005.png', 'S001C001P002R001A007_rgb/061.png', 'S001C001P002R001A007_rgb/038.png', 'S001C001P002R001A007_rgb/066.png', 'S001C001P002R001A007_rgb/075.png', 'S001C001P002R001A007_rgb/071.png', 'S001C001P002R001A007_rgb/001.png', 'S001C001P002R001A007_rgb/064.png', 'S001C001P002R001A007_rgb/014.png', 'S001C001P002R001A007_rgb/054.png', 'S001C001P002R001A007_rgb/003.png', 'S001C001P002R001A007_rgb/011.png', 'S001C001P002R001A007_rgb/033.png', 'S001C001P002R001A007_rgb/056.png', 'S001C001P002R001A007_rgb/024.png', 'S001C001P002R001A007_rgb/058.png', 'S001C001P002R001A007_rgb/022.png', 'S001C001P002R001A007_rgb/052.png', 'S001C001P002R001A007_rgb/045.png', 'S001C001P002R001A007_rgb/065.png', 'S001C001P002R001A007_rgb/057.png', 'S001C001P002R001A007_rgb/0

In [9]:
import os
from glob2 import glob
from tqdm import tqdm

fns = glob('/scratch/PI/cqf/har_data/ntu/nturgb+d_psm2/*/*.png')

for fn in tqdm(fns):
    par_dir = os.path.dirname(fn)
    par_dir_new = par_dir.replace('nturgb+d_psm2', 'nturgb+d120_psm')
    os.makedirs(par_dir_new, exist_ok=True)
    os.system(f'cp {fn} {par_dir_new}')

  0%|          | 973/4780958 [02:06<164:36:03,  8.07it/s]

In [1]:
import os
dirs = os.listdir('/scratch/PI/cqf/har_data/ntu/nturgb+d120_psm')
dirs2 = os.listdir('/scratch/PI/cqf/har_data/ntu/nturgb+d_psm2')

In [2]:
from tqdm import tqdm
for d in tqdm(dirs):
    if d in dirs2:
        os.system(f'rm -r /scratch/PI/cqf/har_data/ntu/nturgb+d_psm2/{d}')

  0%|          | 0/57379 [00:00<?, ?it/s]

100%|██████████| 57379/57379 [00:46<00:00, 1225.38it/s]


In [5]:
for d in tqdm(dirs):
    os.makedirs(f'/scratch/PI/cqf/har_data/ntu/nturgb+d_psm2/{d}', exist_ok=True)

100%|██████████| 57379/57379 [01:02<00:00, 923.92it/s] 


In [3]:
from tqdm import tqdm
for d in tqdm(dirs):
    os.system(f'cp -r /scratch/PI/cqf/har_data/ntu/nturgb+d120_psm/{d} /scratch/PI/cqf/har_data/ntu/nturgb+d_psm2')

  0%|          | 61/57379 [00:42<24:54:51,  1.56s/it]

In [ ]:
from multiprocessing import Pool
def run(d):
    os.system(f'cp -r /scratch/PI/cqf/har_data/ntu/nturgb+d120_psm/{d} /scratch/PI/cqf/har_data/ntu/nturgb+d_psm2')

with Pool(16) as p:
    list(tqdm(p.imap(run, dirs), total=len(dirs)))

In [12]:
import torch
import torch.nn.functional as F

B = 2
C = 3
H = 64
W = 64

x1 = 10
y1 = 10
x2 = 20
y2 = 20

features = torch.randn(B, C, H, W)

# batch of bounding boxes
boxes = torch.tensor([[x1, y1, x2, y2] for _ in range(B)])

# compute the size of the bounding boxes
box_h = boxes[:, 3] - boxes[:, 1]
box_w = boxes[:, 2] - boxes[:, 0]

idxs_h = torch.arange(0, box_h[0], 1).float()
idxs_w = torch.arange(0, box_w[0], 1).float()

print(idxs_h.shape)

# compute the grid
grid_h = ((idxs_h.unsqueeze(0) + boxes[:, 1:2]) / H) * 2 - 1
grid_w = ((idxs_w.unsqueeze(0) + boxes[:, 0:1]) / W) * 2 - 1

print(grid_h.shape, grid_w.shape)

prod = torch.stack(torch.meshgrid(grid_h, grid_w), dim=-1)
print(prod.shape)

# grid_h: [B, h]
# grid_w: [B, w]
# grid: [B, h, w, 2]
# transform 1D index to 2D index
# grid_h.unsqueeze(-1): [B, h, 1]
# grid_w.unsqueeze(-1): [B, 1, w]
# grid: [B, h, w, 2]
grid_h_ = grid_h.unsqueeze(-1).expand(-1, -1, len(idxs_w))
grid_w_ = grid_w.unsqueeze(1).expand(-1, len(idxs_h), -1)
print(grid_h_.shape, grid_w_.shape)
grid = torch.stack([grid_h_, grid_w_], dim=-1)

cropped_features = F.grid_sample(features, grid)
print(cropped_features.shape)

torch.Size([10])
torch.Size([2, 10]) torch.Size([2, 10])


RuntimeError: torch.meshgrid: Expected 0D or 1D tensor in the tensor list but got: -0.6875 -0.6562 -0.6250 -0.5938 -0.5625 -0.5312 -0.5000 -0.4688 -0.4375 -0.4062
-0.6875 -0.6562 -0.6250 -0.5938 -0.5625 -0.5312 -0.5000 -0.4688 -0.4375 -0.4062
[ torch.FloatTensor{2,10} ]

In [13]:
x = torch.arange(0, 10, 1).float()
y = torch.arange(0, 10, 1).float()

xx, yy = torch.meshgrid(x, y)

In [19]:
xx/9

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111,
         0.1111],
        [0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222,
         0.2222],
        [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333,
         0.3333],
        [0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444,
         0.4444],
        [0.5556, 0.5556, 0.5556, 0.5556, 0.5556, 0.5556, 0.5556, 0.5556, 0.5556,
         0.5556],
        [0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667,
         0.6667],
        [0.7778, 0.7778, 0.7778, 0.7778, 0.7778, 0.7778, 0.7778, 0.7778, 0.7778,
         0.7778],
        [0.8889, 0.8889, 0.8889, 0.8889, 0.8889, 0.8889, 0.8889, 0.8889, 0.8889,
         0.8889],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000]])

In [15]:
yy

tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [17]:
xx - 5

tensor([[-5., -5., -5., -5., -5., -5., -5., -5., -5., -5.],
        [-4., -4., -4., -4., -4., -4., -4., -4., -4., -4.],
        [-3., -3., -3., -3., -3., -3., -3., -3., -3., -3.],
        [-2., -2., -2., -2., -2., -2., -2., -2., -2., -2.],
        [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.]])

In [18]:
yy - 5

tensor([[-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
        [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
        [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
        [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
        [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
        [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
        [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
        [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
        [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
        [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.]])

In [21]:
import torch
a = torch.arange(0, -1, 1)

RuntimeError: upper bound and larger bound inconsistent with step sign